In [7]:
from random import randrange
import numpy as np
ACTION = {'UP':[0,0,1],'DOWN':[0,0,-1], 'NORTH':[1,0,0], 'SOUTH':[-1,0,0], 'WEST':[0,1,0], 'EAST':[0,-1,0]}
HANGLE = np.pi/4;

In [23]:
class Agent:
    
    x = 0; 
    y = 0;
    z = 0;
    x_lim = 0;
    y_lim = 0;
    z_lim = 0;
    hangle = HANGLE;
    rank = 1;
    m = None;
    current_act = 'UP';
    
    def __init__(self, m, rank):
        self.m = m;
        self.x_lim, self.y_lim, self.z_lim = m.get_lim();
        self.__init_rand_state();
        self.rank = rank;
        
    def __init_rand_state(self):
        self.x = randrange(0, self.x_lim);
        self.y = randrange(0, self.y_lim);
        self.z = randrange(1, self.z_lim);
        
    def get_state(self):
        return [self.x, self.y, self.z];
    
    def set_state(self, x, y, z):
        if x < self.x_lim and x >= 0 and y < self.y_lim and y >=0 and z < self.z_lim and z>= 0:
            self.x = x;
            self.y = y;
            self.z = z;
        
    def move(self,act):
        self.current_act = act;
        act_value = ACTION.get(act);
        self.set_state(self.x+act_value[0],self.y+act_value[1],self.z+act_value[2]);
    
    def get_act(self):
        return self.current_act;
        
    def state_covered(self, x, y):
        return abs(x-self.x)/self.z <= np.tan(self.hangle) and abs(y-self.y)/self.z <= np.tan(self.hangle);
    
    def map_covered(self):
        map_space = self.m.get_map();
        m_covered = [];
        for x in range(self.x_lim):
            for y in range(self.y_lim):
                if map_space[x][y] >= 1:
                    m_covered.append(self.state_covered(x,y))
        return m_covered;
        

In [64]:
class Map_space:
    map_space = np.array([0,0,1,0,1,0]).reshape(2,3)
    #map_space = np.array([0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]).reshape(4,6);
    height = 4;
    
    def get_lim(self):
        return self.map_space.shape[0], self.map_space.shape[1], self.height
    
    def get_map(self):
        return self.map_space

In [65]:
def map_covered_all(agent_list):
    m_covered_all = np.zeros(len(agent_list[0].map_covered()));
    for agent in agent_list:
        m_covered = agent.map_covered();
        for n in range(len(m_covered_all)):
            if m_covered[n]:
                m_covered_all[n] = m_covered_all[n]+1;
    return m_covered_all;

In [66]:
def global_r(agent_list, b):
    num_covered = 0;
    num_overlap = 0;
    m_covered_all = map_covered_all(agent_list);
    for n in m_covered_all:
        if n >= 1:
            num_covered = num_covered+1;
            if n > 1:
                num_overlap = num_overlap+1;
    if num_covered >= b*len(m_covered_all) and num_overlap == 0:
        return 1;
    else:
        return 0;

In [67]:
def get_basis(agent_list, m):
    sum_d = sum(m.get_lim())
    basis = np.zeros([sum_d*len(agent_list),len(ACTION)**len(agent_list)])
    joint_act = 0;

    for n in range(len(agent_list)):
        a = agent_list[n];
        joint_act = joint_act+list(ACTION.keys()).index(a.get_act())*(len(ACTION)**n)
        
    for n in range(len(agent_list)):
        current_state = agent_list[n].get_state();        
        for l in range(len(current_state)):
            basis[n*sum_d+sum(m.get_lim()[0:l])+current_state[l]][joint_act] = 1;           
    
    return basis
    return basis.reshape(1,basis.size)

In [ ]:
gamma = 0.9 #discount factor
alpha = 0.1 #learning rate
epsilon = 0.9 #possibility for taking random action, diminished over time

